In [43]:
from pathlib import Path
import pandas as pd
import json
import networkx as nx
import matplotlib.pyplot as plt
import pdb
import pickle
import numpy as np
from typing_extensions import Annotated
import time

from mbnpy import model, config, trans, variable, brc, branch, cpm, inference

HOME = Path().resolve()

%matplotlib inline

In [44]:
key = 'od1'
thres = 5 # if it takes longer than this, we consider the od pair is disconnected

In [45]:
# load data
cfg = config.Config(HOME.joinpath('C:/Users/Minji Kang/Documents/GitHub/network_reliability/config.json'))

scenario to be added later


# Variable Distribution Setup

This section defines the distribution types and parameters for variables in the Bayesian Network model. The variable names follow specific prefixes that indicate their role:

| Prefix | Meaning                 | Distribution Type     | Parameters / Description                         |
|--------|-------------------------|------------------------|--------------------------------------------------|
| `L`    | Primary delay           | Normal Distribution    | Mean ∈ [30, 40], StdDev ∈ [5, 10]                |
| `N`    | Number of delayed trains| Normal Distribution    | Mean ∈ [2, 10], StdDev ∈ [1, 3]                  |
| `T`    | Total delay time        | Normal Distribution    | Mean ∈ [10, 100], StdDev ∈ [10, 30]              |

In [46]:
rnd_state = np.random.RandomState(42) 

probs = {}
for node in cfg.infra['nodes'].keys():
    if node == "L1":
        mean = rnd_state.uniform(30, 40)
        std = rnd_state.uniform(5, 10)
        probs[node] = {"type": "normal", "mean": round(mean, 2), "std": round(std, 2)}

    elif node.startswith("L"):
        probs[node] = {"type": "normal"}

    elif node == "N1":
        mean = rnd_state.uniform(2, 10)
        std = rnd_state.uniform(1, 3)
        probs[node] = {"type": "normal", "mean": round(mean, 2), "std": round(std, 2)}

    elif node.startswith("N"):
        probs[node] = {"type": "normal"}
    
    elif node == "T1":
        mean = rnd_state.uniform(50, 100)
        std = rnd_state.uniform(10, 30)
        probs[node] = {"type": "normal", "mean": round(mean, 2), "std": round(std, 2)}

    elif node.startswith("T"):
        probs[node] = {"type": "normal"}
    
    else:
        probs[node] = {"type": "unknown"}
# results
from pprint import pprint
pprint(probs)


{'L1': {'mean': 33.75, 'std': 9.75, 'type': 'normal'},
 'L2': {'type': 'normal'},
 'L3': {'type': 'normal'},
 'L4': {'type': 'normal'},
 'N1': {'mean': 7.86, 'std': 2.2, 'type': 'normal'},
 'N2': {'type': 'normal'},
 'N3': {'type': 'normal'},
 'N4': {'type': 'normal'},
 'T1': {'mean': 57.8, 'std': 13.12, 'type': 'normal'},
 'T2': {'type': 'normal'},
 'T3': {'type': 'normal'},
 'T4': {'type': 'normal'}}


# Modeling
<img src='BN.jpg' alt='BN graph' width='400' />

In [47]:
# variables
varis = {}
cpms = {}
for k, v in cfg.infra['nodes'].items():
    varis[k] = variable.Variable(name=k, values = ['f', 's'])
    cpms[k] = cpm.Cpm(variables = [varis[k]], no_child=1,
                        C = np.array([0, 1]).T, p = [probs[k][0], probs[k][1]])

KeyError: 0